In [1]:
# Load environment variables from .env file
%load_ext dotenv
%dotenv

# Create Graph with a state of all messages

In [2]:
from langgraph.graph import add_messages, StateGraph
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.checkpoint.sqlite import SqliteSaver

class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]
    
graph_builder = StateGraph(State)

# Connection to Couchbase's vector store

In [3]:
from langchain_openai import OpenAIEmbeddings
import os
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions
from couchbase.auth import PasswordAuthenticator
from langchain_couchbase import CouchbaseVectorStore

COUCHBASE_CONNECTION_STRING = os.environ["COUCHBASE_CONNECTION_STRING"]
COUCH_USER = os.environ["COUCHBASE_USER"]
COUCH_PASS = os.environ["COUCHBASE_PASS"]
BUCKET_NAME = os.environ["COUCHBASE_BUCKET"]
SCOPE_NAME = os.environ["COUCHBASE_SCOPE"]
COLLECTION_NAME = os.environ["COUCHBASE_COLLECTION"]
SEARCH_INDEX_NAME = os.environ["COUCHBASE_SEARCH_INDEX"]

auth = PasswordAuthenticator(COUCH_USER, COUCH_PASS)
options = ClusterOptions(auth)
cluster = Cluster(COUCHBASE_CONNECTION_STRING, options)
embedding = OpenAIEmbeddings()
embedding.model = "text-embedding-3-small"

vector_store = CouchbaseVectorStore(
    cluster=cluster,
    bucket_name=BUCKET_NAME,
    scope_name=SCOPE_NAME,
    collection_name=COLLECTION_NAME,
    embedding=embedding,
    index_name=SEARCH_INDEX_NAME,
)

# Create Graph instance

In [4]:
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

def chatbot(state: State):
    context = " ".join([message.content for message in state['messages']])  # Use all messages as context
    print(f"Assistant's context: {context}")
    response = llm.generate(context)
    # Add the response to the messages
    print(f"Chatbot response: {response}")
    state['messages'].append(f"Chatbot response: {response}")
    return state

def search_couchbase(state: State):
    query = state['messages'][-1]
    retriever = vector_store.as_retriever()
    results = retriever.invoke(query.content)
    # Add the results to the messages
    state['messages'].append(HumanMessage(f"Search results: {results}"))
    print(f"number of messages in state: {len(state['messages'])}")
    print(f"Message number 1 of state: {state['messages'][0]}")
    print(f"Message number 2 of state: {state['messages'][1]}")
    return state

graph_builder.add_node("vector_search", search_couchbase)
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("vector_search")
graph_builder.add_edge("vector_search", "chatbot")

memory = SqliteSaver.from_conn_string(":memory:")
graph = graph_builder.compile(checkpointer=memory)


# Interaction

In [5]:
config = {"configurable": {"thread_id": "meetup"}}

while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in graph.stream({"messages": ("user", user_input)}, config):
        for value in event.values():
            print("Assistant:", value["messages"][-1])

number of messages in state: 2
Message number 1 of state: content='What is the capital of France?' id='a9a3479b-e0cf-4856-9b4e-56c3225da371'
Message number 2 of state: content='Search results: []'
Assistant: content='Search results: []'
Assistant's context: What is the capital of France? Search results: []
Chatbot response: generations=[[ChatGeneration(text='Hello! How can I assist you today?', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2e54749c-709e-4735-8420-41701ead4308-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17}))], [ChatGeneration(text='Hello! How can I assist you today?', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(con